#Connecting to SQL Server

In [ ]:
%pip install colab_env -qU sshtunnel pymysql sqlalchemy==1.4.46 MySQLclient

**colab_env** to access and edit enviromental variables.

**sshtunnel** allows you to create a connection to an ssh server using python.

**MySQLclient** grants access to all of the tools desigined to connect to MySQL Server.

**pymysql** allows you to perform CRUD (Create, Read, Update, Delete) functions to manage the database.

**sqlachemy** facilitates the communication between Python programs and databases.

In [ ]:
# !more gdrive/My\ Drive/vars.env

In [ ]:
# colab_env.RELOAD()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import colab_env
#from colab_env import envvar_handler #use to create enviroment variables colab_env.envvar_handler.add_env("VAR_NAME", "VALUE", overwrite=True)
import os
import sshtunnel
import logging
import pymysql
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import sqlalchemy
from sqlalchemy import Table, Column, Integer, Float, VARCHAR, String, DateTime, MetaData
import MySQLdb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/gdrive


##Functions and Variables to Access the Server

In [ ]:
#Server Variables
SERVER_IP  = os.getenv('SERVER_IP')
SERVER_PORT = int(os.getenv('SERVER_PORT')) # or ip
SERVER_USER = os.getenv('SERVER_USER')
SERVER_USER_PASSWORD  = os.getenv('SERVER_USER_PASSWORD')

#Database Variables
SQL_SERVER_USER  = os.getenv('SQL_SERVER_USER')
SQL_SERVER_USER_PASSWORD = os.getenv('SQL_SERVER_USER_PASSWORD')
SQL_DB_NAME  = os.getenv('SQL_DB_NAME')
SQL_DP_PORT = int(os.getenv('SQL_DB_PORT'))
LOCAL_HOST = os.getenv('Local_Host')

In [ ]:
#Function to create the connection to the server
def serverConnection(verbose=False):

  if verbose:
    sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG

  global tunnel
  #connects to the server and creates a tunnel connection
  tunnel = SSHTunnelForwarder((SERVER_IP, SERVER_PORT),
                                  ssh_username= SERVER_USER, ssh_password=SERVER_USER_PASSWORD,
                                  remote_bind_address=(LOCAL_HOST, SQL_DP_PORT))

  tunnel.start()

In [ ]:
#Function to establish a connection to the database using PyMySQL
def sqlConnection():

  global connection

  connection = pymysql.connect(host=LOCAL_HOST, user=SQL_SERVER_USER,
                               passwd=SQL_SERVER_USER_PASSWORD, db=SQL_DB_NAME,
                               port=tunnel.local_bind_port)



In [ ]:
#Function that takes a string which will act as the SQL query and returns a dataframe using pandas
def runQueryPyMySQL(sql):
  #returns a panda dataframe of results from query
  return pd.read_sql_query(sql, connection)


In [ ]:
#Function that creates a connection to the DB using SQLAlchemy instead of PyMySQL
#Instead of only being able to return a dataframe. This function can execute any
#-SQL statement.
def runQuerySQLAlchemy(sql=None):
  global engine
  local_port = str(tunnel.local_bind_port)
  engine = sqlalchemy.create_engine(
  f'mysql+mysqldb://{SQL_SERVER_USER}:{SQL_SERVER_USER_PASSWORD}@{LOCAL_HOST}:{local_port}/{SQL_DB_NAME}')

  if sql == None:
    pass
  else:
    return engine.execute(sql)



In [ ]:
#Function that closes the connection to the SQL Database
def closeSQLConnection():
  connection.close()

In [ ]:
#Function that closes the connection the server
def closeServerConnection():
  tunnel.close

##Executing SQL Commands

In [ ]:
serverConnection() #Creates server connection

In [ ]:
sqlConnection() #Creates database connection using PyMySQL

In [ ]:
runQuerySQLAlchemy() #Creates database connection using SQLAlchemy engine

In [ ]:
temp = engine.execute("SHOW TABLES").fetchall()


temp

  temp = engine.execute("SHOW TABLES").fetchall()



[('Completers',),
 ('NYS',),
 ('NYSCTE',),
 ('PSTE',),
 ('Students',),
 ('Taskstream',)]

In [ ]:
##Inititalizes metadata object that stores the information of the table or any object that references it
metadata_obj = MetaData()


In [ ]:
# #Creates Taskstream table
# #When creating tables be mindful that column names must match the the column of the dataframe
# Taskstream = Table(
#     "Taskstream", metadata_obj,
#     Column("StudentID", String(16), nullable=False),
#     Column("ProgramName", String(255)),
#     Column("Name", String(255)),
#     Column("CriteriaName", VARCHAR(1050)),
#     Column("RubricRowPoints", Float()),
#     Column("timestamp", DateTime()),
# )

# Completers = Table(
#     "Completers", metadata_obj,
#     Column("BANNER_ID", String(16), nullable=False),
#     Column("DEGREE_MAJORid", String(20)),
#     Column("GRAD_TERM", Integer()),
#     Column("COHORT", String(25)),
#     Column("AdvInit", String(15)),
#     Column("ProgGroup", String(255)),
#     Column("ProgAbbrev", String(50)),
#     Column("ProgLevel", String(5)),
# )

# uniqueCompleters = Table(
#     "Students", metadata_obj,
#     Column("BANNER_ID", String(16), nullable=False, primary_key=True),
#     Column("ETHNICITY", String(20)),
#     Column("GENDER", String(16)),
# )

In [ ]:
##The engine executes all of the objects that are stored within the metadata object
# metadata_obj.create_all(engine)

Append data to sql table

In [ ]:
# taskstreamDF = pd.read_csv('/content/drive/MyDrive/R Manuel Work Study Folder/Output Files/Taskstream.csv')
# completersDF = pd.read_csv('/content/gdrive/MyDrive/R Manuel Work Study Folder/Output Files/221026completers.csv')
# uniqueCompletersDF = pd.read_csv('/content/drive/MyDrive/R Manuel Work Study Folder/Output Files/uniqueCompleters.csv')

In [ ]:
##Completers dataframe has not been added yet

## To repalce data in table use code below
# DATAFRAME.to_sql("TABLE NAME", engine, if_exists="replace", index=False)

##Data was already addded to database. If adding data to existing database use "append"

# uniqueCompletersDF.to_sql("Students", engine, if_exists="append", index=False)
# taskstreamDF.to_sql("Taskstream", engine, if_exists="append", index=False)
# completersDF.to_sql("Completers", engine, if_exists="append", index=False)

Rename table names ask Tiffany

In [ ]:
# engine.execute("ALTER TABLE x RENAME y TO z;")

##Using pandas to Read Table Data

Once the data is on the server pandas has a built in function that use the engine created after running the runQuerySQLAlchemy to read the data directly into a dataframe.

In [ ]:
df = pd.read_sql_table('Taskstream', engine)

In [ ]:
df.head()

,StudentID,FirstName,LastName,ProgramName,Name,CriteriaName,RubricRowPoints,timestamp
0,H05910,zachary,boehler,"Art Education, Advanced",CAEP Advanced Programs Capstone 690 Rubric (cu...,Evidence based contribution to the teaching pr...,4.0,2020-01-22 09:55:01
1,H05910,zachary,boehler,"Art Education, Advanced",CAEP Advanced Programs Capstone 690 Rubric (cu...,Specialized pedagogical knowledge CAEP A.1.2 ...,3.0,2020-01-22 09:55:01
2,H05910,zachary,boehler,"Art Education, Advanced",CAEP Advanced Programs Capstone 690 Rubric (cu...,Impact on P-12 student-learning growth and / o...,4.0,2020-01-22 09:55:01
3,H05910,zachary,boehler,"Art Education, Advanced",CAEP Advanced Programs Capstone 690 Rubric (cu...,Professional Responsibility CAEP A.1.1 (Collab...,4.0,2020-01-22 09:55:01
4,H05910,zachary,boehler,"Art Education, Advanced",CAEP Advanced Programs Capstone 690 Rubric (cu...,"Technology CAEP A.1.1 (Tech, Data, Lit) CAEP A...",3.5,2020-01-22 09:55:01


In [ ]:
df2 = pd.read_sql_table('Students', engine)

In [ ]:
df2.head()

,BANNER_ID,ETHNICITY,GENDER
0,H010010,B - Black,F
1,H01008,W - White,F
2,H010101,H - Hispanic,M
3,H0101010,A - Asian,F
4,H0101011,B - Black,F


In [ ]:
closeSQLConnection() #Only needs to be used when using runQueryPyMySQL() or when reading the table directly using pandas

In [ ]:
closeServerConnection() #Closes the connection to the server

##Check if table exits before adding to the database
Look through code
https://stackoverflow.com/questions/33053241/sqlalchemy-if-table-does-not-exist

In [ ]:
# if not engine.dialect.has_table(engine, Variable_tableName):  # If table does not exist, Create.
#     metadata = MetaData(engine)
#     # Create a table with the appropriate Columns
#     Table(Variable_tableName, metadata,
#           Column('Column Name', Data Type, primary_key=True, nullable=False),
#           Column('Column Name', Data Type),
#           Column('Column Name', Data Type),
#           Column('Column Name', Data Type),
#           Column('Column Name', Data Type),
#     # Implement the creation
#     metadata.create_all(engine)

In [ ]:
# df = runQueryPyMySQL("SELECT * FROM Completers")
# df.head()

#SQLAlchemy Documentation

https://docs.sqlalchemy.org/en/14/core/engines.html

https://docs.sqlalchemy.org/en/14/core/metadata.html

#Read CSV to SQL Server

https://www.andrewvillazon.com/move-data-to-db-with-sqlalchemy/

# Save Server.py File

In [ ]:
%%writefile "/content/gdrive/MyDrive/R Manuel Work Study Folder/database/Server2.py"

def installpackages():
  import subprocess
  import sys

  packages = ["colab_env", "sshtunnel", "pymysql", "sqlalchemy==1.4.46", "MySQLclient"]
  for x in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", x])


def createVariables():
  #Server Variables
  globals() ["SERVER_IP"]  = os.environ['SERVER_IP']
  globals() ["SERVER_PORT"] = int(os.environ['SERVER_PORT']) # or ip
  globals() ["SERVER_USER"] = os.environ['SERVER_USER']
  globals() ["SERVER_USER_PASSWORD"] = os.environ['SERVER_USER_PASSWORD']
  #Database variables
  globals() ["SQL_SERVER_USER"]  = os.environ['SQL_SERVER_USER']
  globals() ["SQL_SERVER_USER_PASSWORD"] = os.environ['SQL_SERVER_USER_PASSWORD']
  #globals() ["SQL_DB_NAME"]  = os.environ['SQL_DB_NAME']
  globals() ["SQL_DP_PORT"] = int(os.environ['SQL_DB_PORT'])
  globals() ["LOCAL_HOST"] = os.environ['Local_Host']


def serverConnection(verbose=False):

  if verbose:
    sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG

  global tunnel
  #connects to the server and creates a tunnel connection
  tunnel = SSHTunnelForwarder((SERVER_IP, SERVER_PORT),
                                  ssh_username= SERVER_USER, ssh_password=SERVER_USER_PASSWORD,
                                  remote_bind_address=(LOCAL_HOST, SQL_DP_PORT))

  tunnel.start()

#Function to establish a connection to the database using PyMySQL
def sqlConnection(DB_Name):

  globals() ['DB_Name'] = DB_Name

  globals() ['connection'] = pymysql.connect(host=LOCAL_HOST, user=SQL_SERVER_USER,
                               passwd=SQL_SERVER_USER_PASSWORD, db=DB_Name,
                               port=tunnel.local_bind_port)


#Function that takes a string which will act as the SQL query and returns a dataframe using pandas
def runQueryPyMySQL(sql):
  local_port = str(tunnel.local_bind_port)

  globals() ['engine'] = sqlalchemy.create_engine(
  f'mysql+mysqldb://{SQL_SERVER_USER}:{SQL_SERVER_USER_PASSWORD}@{LOCAL_HOST}:{local_port}/{DB_Name}')

  #returns a panda dataframe of results from query
  return pd.read_sql_query(sql, engine)

#Function that creates a connection to the DB using SQLAlchemy instead of PyMySQL
#Instead of only being able to return a dataframe. This function can execute any
#-SQL statement.
def runQuerySQLAlchemy(sql=None):
  local_port = str(tunnel.local_bind_port)

  globals() ['engine'] = sqlalchemy.create_engine(
  f'mysql+mysqldb://{SQL_SERVER_USER}:{SQL_SERVER_USER_PASSWORD}@{LOCAL_HOST}:{local_port}/{DB_Name}')

  if sql == None:
    pass
  else:
    return engine.execute(sql).fetchall()

#Function that closes the connection to the SQL Database
def closeSQLConnection():
  connection.close()

#Function that closes the connection the server
def closeServerConnection():
  tunnel.close


installpackages()

import os
import colab_env
import sshtunnel
import logging
import pymysql
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import sqlalchemy
from sqlalchemy import Table, Column, Integer, Float, VARCHAR, String, DateTime, MetaData
import MySQLdb


createVariables()

Overwriting /content/gdrive/MyDrive/R Manuel Work Study Folder/database/Server2.py
